In [ ]:
from pop import TempHumi
import struct
import time

def write_bitmap_header(f, bfSize):  
    f.write(struct.pack("H", 0x4D42))
    f.write(struct.pack("I", bfSize))
    f.write(struct.pack("H", 0))
    f.write(struct.pack("H", 0))
    f.write(struct.pack("I", 54))

def write_bitmap_info_header(f, biWidth, biHeight, biByteCount, biSizeImage):
    f.write(struct.pack("I", 40))
    f.write(struct.pack("I", biWidth))
    f.write(struct.pack("I", biHeight))
    f.write(struct.pack("H", 1))
    f.write(struct.pack("H", biByteCount * 8))
    f.write(struct.pack("I", 0))
    f.write(struct.pack("I", biSizeImage))
    f.write(struct.pack("I", 0))
    f.write(struct.pack("I", 0))
    f.write(struct.pack("I", 0))
    f.write(struct.pack("I", 0))

def set_rgb(array, base, n):
    array[base+0] = (n >> 0) & 0xFF
    array[base+1] = (n >> 8) & 0xFF
    array[base+2] = (n >> 16) & 0xFF
    
th = TempHumi()

with open("sensorArt.bmp", "bw") as f:     
    biWidth = 20
    biHeight = 20
    biByteCount = 3
    biSizeImage = biWidth * biHeight * biByteCount
    bfSize = biSizeImage + 54

    write_bitmap_header(f, bfSize)
    write_bitmap_info_header(f, biWidth, biHeight, biByteCount, biSizeImage)

    data = bytearray((" " * biSizeImage).encode())
    for i in range(biHeight):
        for j in range(0, biWidth * 3, 3):
            valTemp = int(th.getTemperature())
            valHumi = int(th.getHumidity())
            
            print("Temp: %d, Humi: %d" % (valTemp, valHumi))

            r = valTemp & 0xFF
            g = ((valTemp >> 8) & 0xFF) | ((valHumi >> 8) & 0xFF)
            b = valHumi & 0xFF

            pixel = (r << 16 | g << 8 | b) & 0xFFFFFF
            set_rgb(data, i * (biWidth * 3) + j, pixel)
            
            time.sleep(0.01)

    f.write(data)
